# The NammaMetro Network

Namma Metro is Bangalore’s rapid transit system, operated by Bangalore Metro Rail Corporation Limited (BMRCL), a joint venture between the Indian government and Karnataka state. It’s the second-largest metro network in India—after Delhi Metro—with an operational length of 96.1 km, and 83 stations, including elevated, underground, and at-grade stations.

Ridership: The average daily ridership was 7,62,361 in July 2024, with a record high of 10,48,031 on 11 August 2025. Annual ridership in 2023 stood at 232.8 million.

Network: The operational lines are Purple, Green, and Yellow, with Pink and Blue lines under construction, and Orange, Grey, Red, and Inner Ring lines proposed or approved. The system runs on standard gauge track with 750V DC electrification via a third rail.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io, requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_Namma_Metro_stations"
headers = {
    "User-Agent": "WikiTablesFetcher/1.0 (contact: ms@thecontrarian.in)",
    "Accept-Language": "en-US,en;q=0.9"
}

resp = requests.get(url, headers=headers, timeout=30)
resp.raise_for_status()

NammaMetro_Stations = pd.read_html(io.StringIO(resp.text), flavor="lxml", displayed_only=True)

NammaMetro_Stations_1 = NammaMetro_Stations[1].loc[:, [('Line', 'Line'), 
                                                       ('Abbreviations', 'Abbreviations'), 
                                                       ('Station name', 'English'), 
                                                       ('Station name', 'Kannada')]].copy()
NammaMetro_Stations_4 = NammaMetro_Stations[4].loc[:, [('Line', 'Line'), 
                                                       ('Abbreviations', 'Abbreviations'), 
                                                       ('Station name', 'English'), 
                                                       ('Station name', 'Kannada')]].copy()
NammaMetro_Stations = pd.concat([NammaMetro_Stations_1, NammaMetro_Stations_4], ignore_index=True)

NammaMetro_Stations.columns = [f'{a}' for a,b in NammaMetro_Stations.columns]
NammaMetro_Stations.columns = ['line_name', 'code', 'name_eng', 'name_kan']
NammaMetro_Stations['line_name'] = NammaMetro_Stations['line_name'].str.split().str[0]
NammaMetro_Stations['code'] = NammaMetro_Stations['code'].str.replace('TBC', '[TBC]')

line_colours = {'Purple': '🟣',  'Green': '🟢', 'Yellow': '🟡', 'Pink'  : '🌸', 
                'Blue': '🔵', 'Orange': '🟠',    'Red': '🔴', 'Grey': '⚪'}
NammaMetro_Stations['line_colour'] = NammaMetro_Stations['line_name'].map(line_colours)

lines = NammaMetro_Stations.groupby(['name_eng'])['line_colour'].sum().copy()
NammaMetro_Stations['lines'] = NammaMetro_Stations['name_eng'].map(lines)
NammaMetro_Stations = NammaMetro_Stations[['code', 'name_eng', 'name_kan', 'lines']].drop_duplicates()
NammaMetro_Stations['is_terminus'] = NammaMetro_Stations['name_eng'].str.contains('\\*')
NammaMetro_Stations['is_interchange'] = NammaMetro_Stations['name_eng'].str.contains('†')
NammaMetro_Stations['name_eng'] = NammaMetro_Stations['name_eng'].str.split('†').str[0].str.split('*').str[0].str.strip()

NammaMetro_Stations = NammaMetro_Stations.sort_values('name_eng').reset_index(drop=True)
display(NammaMetro_Stations)

NammaMetro_Stations.to_csv("NammaMetro_Stations_Dataset.csv", index=False)

,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,[TBC],Agara,ಅಗರ,🔵🔴,False,True
1,[TBC],Airport City,ವಿಮಾನ ನಿಲ್ದಾಣ ನಗರ,🔵,False,False
2,[TBC],Ambedkar Nagar,ಅಂಬೇಡ್ಕರ್ ನಗರ,🔴,False,False
3,AGPP,Attiguppe,ಅತ್ತಿಗುಪ್ಪೆ,🟣,False,False
4,[TBC],BEL Circle,ಬಿ ಇ ಎಲ್ ವೃತ್ತ,🟠,False,False
...,...,...,...,...,...,...
167,[TBC],Vinayaka Nagar,ವಿನಾಯಕ ನಗರ,⚪,False,False
168,WHTM,Whitefield (Kadugodi),ವೈಟ್ ಫೀಲ್ಡ್ (ಕಾಡುಗೋಡಿ),🟣,True,False
169,PUTH,Yelachenahalli,ಯಲಚೇನಹಳ್ಳಿ,🟢,False,False
170,[TBC],Yelahanka,ಯಲಹಂಕ,🔵,False,False


In [3]:
print(f"Number of functioning stations: {len(NammaMetro_Stations[~(NammaMetro_Stations['code'] == '[TBC]')])}")
print(f"Number of stations under construction: {len(NammaMetro_Stations[(NammaMetro_Stations['code'] == '[TBC]')])}")
for line in line_colours.values():
    print(f"Number of {line} stations: {len(NammaMetro_Stations[(NammaMetro_Stations['lines'].str.contains(line))])}")

Number of functioning stations: 83
Number of stations under construction: 89
Number of 🟣 stations: 37
Number of 🟢 stations: 32
Number of 🟡 stations: 16
Number of 🌸 stations: 18
Number of 🔵 stations: 29
Number of 🟠 stations: 19
Number of 🔴 stations: 28
Number of ⚪ stations: 8


In [4]:
columns = ['code', 'name_eng', 'name_kan', 'lines']

print("\nTERMINUS STATIONS")
display(NammaMetro_Stations[columns][NammaMetro_Stations['is_terminus']].sort_index())

print("INTERCHANGE STATIONS")
display(NammaMetro_Stations[columns][NammaMetro_Stations['is_interchange']].sort_index())


TERMINUS STATIONS


,code,name_eng,name_kan,lines
23,CLGA,Challaghatta,ಚಲ್ಲಘಟ್ಟ,🟣
31,BMSD,Delta Electronics Bommasandra,ಡೆಲ್ಟಾ ಎಲೆಕ್ಟ್ರಾನಿಕ್ಸ್ ಬೊಮ್ಮಸಂದ್ರ,🟡
50,[TBC],Hebbala,ಹೆಬ್ಬಾಳ,🔵🟠🔴
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠
75,[TBC],KIAL Terminals,ಕೆಂಪೇಗೌಡ ಅಂತಾರಾಷ್ಟ್ರೀಯ ವಿಮಾನ ನಿಲ್ದಾಣ,🔵
82,[TBC],Kalena Agrahara,ಕಾಳೇನ ಅಗ್ರಹಾರ,🌸
87,[TBC],Kempapura,ಕೆಂಪಾಪುರ,🔵🟠
105,BIET,Madavara,ಮಾದಾವರ,🟢
122,[TBC],Nagawara,ನಾಗವಾರ,🌸🔵
137,RVR,Rashtreeya Vidyalaya Road,ರಾಷ್ಟ್ರೀಯ ವಿದ್ಯಾಲಯ ರಸ್ತೆ,🟢🟡


INTERCHANGE STATIONS


,code,name_eng,name_kan,lines
0,[TBC],Agara,ಅಗರ,🔵🔴
22,SBJT,Central Silk Board,ಕೇಂದ್ರ ರೇಷ್ಮೆ ಮಂಡಳಿ,🟡🔵
28,[TBC],Dairy Circle,ಡೈರಿ ವೃತ್ತ,🌸🔴
50,[TBC],Hebbala,ಹೆಬ್ಬಾಳ,🔵🟠🔴
62,[TBC],Ibbaluru,ಇಬ್ಬಲೂರು,🔵🔴
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠
71,JDHP,Jayadeva Hospital,ಜಯದೇವ ಆಸ್ಪತ್ರೆ,🟡🌸
87,[TBC],Kempapura,ಕೆಂಪಾಪುರ,🔵🟠
117,KGWA,"Nadaprabhu Kempegowda Station, Majestic","ನಾಡಪ್ರಭು ಕೆಂಪೇಗೌಡ ನಿಲ್ದಾಣ, ಮೆಜೆಸ್ಟಿಕ್",🟣🟢
122,[TBC],Nagawara,ನಾಗವಾರ,🌸🔵


In [5]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🟣')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
3,AGPP,Attiguppe,ಅತ್ತಿಗುಪ್ಪೆ,🟣,False,False
9,BYPL,Baiyappanahalli,ಬೈಯ್ಯಪ್ಪನಹಳ್ಳಿ,🟣,False,False
15,JTPM,Benniganahalli,ಬೆನ್ನಿಗಾನಹಳ್ಳಿ,🟣,False,False
23,CLGA,Challaghatta,ಚಲ್ಲಘಟ್ಟ,🟣,True,False
27,CBPK,Cubbon Park,ಕಬ್ಬನ್ ಪಾರ್ಕ್,🟣,False,False
30,DJNR,Deepanjali Nagar,ದೀಪಾಂಜಲಿ ನಗರ,🟣,False,False
38,VDSA,"Dr. B. R. Ambedkar Station, Vidhana Soudha","ಡಾ. ಬಿ. ಆರ್. ಅಂಬೇಡ್ಕರ್ ನಿಲ್ದಾಣ, ವಿಧಾನ ಸೌಧ",🟣,False,False
44,GDCP,Garudacharpalya,ಗರುಡಾಚಾರ್‍‍ಪಾಳ್ಯ,🟣,False,False
49,HLRU,Halasuru,ಹಲಸೂರು,🟣,False,False
53,DKIA,Hoodi,ಹೂಡಿ,🟣,False,False


In [6]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🟢')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
10,BSNK,Banashankari,ಬನಶಂಕರಿ,🟢,False,False
24,CKPE,Chickpete,ಚಿಕ್ಕಪೇಟೆ,🟢,False,False
25,CKBD,Chikkabidarakallu,ಚಿಕ್ಕಬಿದರಕಲ್ಲು,🟢,False,False
29,DSH,Dasarahalli,ದಾಸರಹಳ್ಳಿ,🟢,False,False
33,KLPK,Doddakallasandra,ದೊಡ್ಡಕಲ್ಲಸಂದ್ರ,🟢,False,False
45,YPI,Goraguntepalya,ಗೊರಗುಂಟೆಪಾಳ್ಯ,🟢,False,False
69,JLHL,Jalahalli,ಜಾಲಹಳ್ಳಿ,🟢,False,False
70,JPN,Jaya Prakash Nagar,ಜಯಪ್ರಕಾಶ ನಗರ,🟢,False,False
72,JYN,Jayanagar,ಜಯನಗರ,🟢,False,False
93,APRC,Konanakunte Cross,ಕೋಣನಕುಂಟೆ ಕ್ರಾಸ್,🟢,False,False


In [7]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🟡')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
5,BTML,BTM Layout,ಬಿ ಟಿ ಎಂ ಬಡಾವಣೆ,🟡,False,False
16,HOSR,Beratena Agrahara,ಬೆರಟೇನ ಅಗ್ರಹಾರ,🟡,False,False
17,HBGI,Biocon Hebbagodi,ಬಯೋಕಾನ್ ಹೆಬ್ಬಗೋಡಿ,🟡,False,False
18,HSRL,Bommanahalli,ಬೊಮ್ಮನಹಳ್ಳಿ,🟡,False,False
22,SBJT,Central Silk Board,ಕೇಂದ್ರ ರೇಷ್ಮೆ ಮಂಡಳಿ,🟡🔵,False,True
31,BMSD,Delta Electronics Bommasandra,ಡೆಲ್ಟಾ ಎಲೆಕ್ಟ್ರಾನಿಕ್ಸ್ ಬೊಮ್ಮಸಂದ್ರ,🟡,True,False
39,ETCT,Electronic City,ಎಲೆಕ್ಟ್ರಾನಿಕ್ ಸಿಟಿ,🟡,False,False
52,OFDC,Hongasandra,ಹೊಂಗಸಂದ್ರ,🟡,False,False
56,BSRD,Hosa Road,ಹೊಸ ರಸ್ತೆ,🟡,False,False
59,HSKR,Huskur Road,ಹುಸ್ಕೂರು ರಸ್ತೆ,🟡,False,False


In [8]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🌸')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
20,[TBC],Cantonment Railway Station,ದಂಡು ರೈಲ್ವೇ ನಿಲ್ದಾಣ,🌸,False,False
28,[TBC],Dairy Circle,ಡೈರಿ ವೃತ್ತ,🌸🔴,False,True
58,[TBC],Hulimavu,ಹುಳಿಮಾವು,🌸,False,False
60,[TBC],IIMB,ಐ ಐ ಎಂ ಬಿ,🌸,False,False
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠,True,True
71,JDHP,Jayadeva Hospital,ಜಯದೇವ ಆಸ್ಪತ್ರೆ,🟡🌸,False,True
80,[TBC],Kadugundanahalli,ಕಾಡುಗುಂಡನಹಳ್ಳಿ,🌸,False,False
82,[TBC],Kalena Agrahara,ಕಾಳೇನ ಅಗ್ರಹಾರ,🌸,True,False
102,[TBC],Lakkasandra,ಲಕ್ಕಸಂದ್ರ,🌸,False,False
104,[TBC],Langford Town,ಲ್ಯಾಂಗ್ಫೋರ್ಡ್ ಟೌನ್,🌸,False,False


In [9]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🔵')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,[TBC],Agara,ಅಗರ,🔵🔴,False,True
1,[TBC],Airport City,ವಿಮಾನ ನಿಲ್ದಾಣ ನಗರ,🔵,False,False
6,[TBC],Bagalur Cross,ಬಾಗಲೂರು ಕ್ರಾಸ್,🔵,False,False
7,[TBC],Bagmane Developers - DRDO Sports Complex,ರ.ಸಂ.ಅ.ಸಂ. ಕ್ರೀಡಾ ಸಂಕೀರ್ಣ,🔵,False,False
12,[TBC],Bellandur,ಬೆಳ್ಳಂದೂರು,🔵,False,False
22,SBJT,Central Silk Board,ಕೇಂದ್ರ ರೇಷ್ಮೆ ಮಂಡಳಿ,🟡🔵,False,True
32,[TBC],Doddajala,ದೊಡ್ಡಜಾಲ,🔵,False,False
35,[TBC],Doddanekundi,ದೊಡ್ಡನೆಕ್ಕುಂದಿ,🔵,False,False
40,[TBC],Embassy TechVillage Kadubeesanahalli,ಕಾಡುಬೀಸನಹಳ್ಳಿ,🔵,False,False
46,[TBC],HBR Layout,ಎಚ್‌ ಬಿ ಆರ್ ಬಡಾವಣೆ,🔵,False,False


In [10]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🟠')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
4,[TBC],BEL Circle,ಬಿ ಇ ಎಲ್ ವೃತ್ತ,🟠,False,False
8,[TBC],Bahubhali Nagara,ಬಾಹುಬಲಿ ನಗರ,🟠,False,False
26,[TBC],Chowdeshwari Nagara,ಚೌಡೇಶ್ವರಿ ನಗರ,🟠,False,False
37,[TBC],Dr. Ambedkar Institute of Technology,ಡಾ||ಅಂಬೇಡ್ಕರ್‌ ತಾಂತ್ರಿಕ ಮಹಾವಿದ್ಯಾಲಯ,🟠,False,False
42,[TBC],Freedom Fighters Cross,ಫ್ರೀಡಂ ಫೈಟರ್ಸ್ ಕ್ರಾಸ್,🟠,False,False
50,[TBC],Hebbala,ಹೆಬ್ಬಾಳ,🔵🟠🔴,True,True
57,[TBC],Hosakerahalli Cross,ಹೊಸಕೆರೆಹಳ್ಳಿ ಕ್ರಾಸ್,🟠,False,False
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠,True,True
66,[TBC],JP Nagar 5th Phase,ಜೆಪಿ ನಗರ ೫ನೇ ಹಂತ,🟠,False,False
78,[TBC],Kadirenahalli Cross,ಕದಿರೇನಹಳ್ಳಿ ಕ್ರಾಸ್,🟠,False,False


In [11]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('🔴')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,[TBC],Agara,ಅಗರ,🔵🔴,False,True
2,[TBC],Ambedkar Nagar,ಅಂಬೇಡ್ಕರ್ ನಗರ,🔴,False,False
11,[TBC],Basaveshwara Circle,ಬಸವೇಶ್ವರ ವೃತ್ತ,🔴,False,False
13,[TBC],Bellandur Gate,ಬೆಳ್ಳಂದೂರು ಗೇಟ್,🔴,False,False
14,[TBC],Bengaluru Golf Course,ಬೆಂಗಳೂರು ಗಾಲ್ಫ್ ಕೋರ್ಸ್,🔴,False,False
21,[TBC],Carmelaram,ಕಾರ್ಮೆಲರಾಮ್,🔴,False,False
28,[TBC],Dairy Circle,ಡೈರಿ ವೃತ್ತ,🌸🔴,False,True
34,[TBC],Doddakannelli,ದೊಡ್ಡಕನ್ನೆಲ್ಲಿ,🔴,False,False
36,[TBC],Dommasandra,ದೊಮ್ಮಸಂದ್ರ,🔴,False,False
43,[TBC],Ganganagar,ಗಂಗಾನಗರ,🔴,False,False


In [12]:
display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains('⚪')])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
19,[TBC],Byadarahalli,ಬ್ಯಾಡರಹಳ್ಳಿ,⚪,False,False
41,[TBC],Forest Gate,ಅರಣ್ಯ ದ್ವಾರ,⚪,False,False
51,[TBC],Herohalli,ಹೇರೋಹಳ್ಳಿ,⚪,False,False
74,[TBC],KHB Colony,ಕೆ ಎಚ್ ಬಿ ಕಾಲೋನಿ,⚪,False,False
77,[TBC],Kadabagere,ಕಡಬಗೆರೆ,⚪,False,False
153,[TBC],Sumanahalli Cross,ಸುಮನಹಳ್ಳಿ ಕ್ರಾಸ್,🟠⚪,False,False
154,[TBC],Sunkadakatte,ಸುಂಕದಕಟ್ಟೆ,⚪,False,False
167,[TBC],Vinayaka Nagar,ವಿನಾಯಕ ನಗರ,⚪,False,False
